In [16]:
import tensorflow as tf

gpus = tf.config.list_physical_devices('GPU')
if gpus:
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)
    

In [17]:
import pandas as pd
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.preprocessing import StandardScaler,LabelEncoder
from sklearn.pipeline import Pipeline
from scikeras.wrappers import KerasClassifier
import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Input
from tensorflow.keras.callbacks import EarlyStopping

In [18]:
df=pd.read_csv("https://raw.githubusercontent.com/Arannamoy/datasets/refs/heads/main/Churn_Modelling/Churn_Modelling.csv")

In [19]:
df=df.drop(['RowNumber','CustomerId','Surname'],axis=1)

In [20]:
label_encoder=LabelEncoder()
df['Gender']=label_encoder.fit_transform(df['Gender'])

df['Geography']=label_encoder.fit_transform(df['Geography'])



In [21]:
df.head(15)

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,0,0,42,2,0.00,1,1,1,101348.88,1
1,608,2,0,41,1,83807.86,1,0,1,112542.58,0
2,502,0,0,42,8,159660.80,3,1,0,113931.57,1
3,699,0,0,39,1,0.00,2,0,0,93826.63,0
4,850,2,0,43,2,125510.82,1,1,1,79084.10,0
5,645,2,1,44,8,113755.78,2,1,0,149756.71,1
6,822,0,1,50,7,0.00,2,1,1,10062.80,0
7,376,1,0,29,4,115046.74,4,1,0,119346.88,1
8,501,0,1,44,4,142051.07,2,0,1,74940.50,0
9,684,0,1,27,2,134603.88,1,1,1,71725.73,0


In [22]:
X=df.drop('Exited',axis=1)
y=df['Exited']

In [23]:
X_train,X_test,y_train,y_test=train_test_split(X,y,random_state=42,test_size=0.25)

In [24]:
scaler=StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)

In [25]:
import pickle

In [26]:
with open("label_encoder.pkl","wb") as fl:
    pickle.dump(label_encoder,fl)

with open("scaler.pkl","wb") as fl:
    pickle.dump(scaler,fl)

In [27]:
### Define a function to create a model and try different parameter

def create_model(neurons=32,layers=1):
    tf.keras.backend.clear_session()
    model=Sequential()
    # model.add(Input(shape=(X_train.shape[1],)))
    # model.add(Dense(neurons,activation=tf.keras.activations.relu))
    model.add(Dense(neurons,activation='relu',input_shape=(X_train.shape[1],)))
    for _ in range(layers-1):
        model.add(Dense(neurons,activation=tf.keras.activations.relu))

    model.add(Dense(1,activation=tf.keras.activations.sigmoid))
    model.compile(optimizer=tf.optimizers.Adam(learning_rate=0.01),loss=tf.losses.BinaryCrossentropy(),metrics=['accuracy'])

    return model    

In [28]:
# Create a keras classifier
model=KerasClassifier(layers=1,neurons=32,build_fn=create_model,epochs=50,batch_size=10,verbose=0)

In [29]:
# define the grid search parameter

param_grid={
    "neurons":[16,32,64,128],
    "layers":[1,2,3],
    "batch_size":[10,20],
    "epochs":[50,100]
}

In [ ]:
grid=GridSearchCV(estimator=model,param_grid=param_grid,n_jobs=1,cv=3)
grid_result=grid.fit(X_train,y_train)

print("Best %f using %s"% (grid_result.best_score_,grid_result.best_params_))

/home/workstation-p/anaconda3/envs/conda-env-3-12/lib/python3.12/site-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/home/workstation-p/anaconda3/envs/conda-env-3-12/lib/python3.12/site-packages/keras/src/layers/core/dense.py:106: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
I0000 00:00:1771086795.719965   54572 cuda_executor.cc:508] failed to allocate 27.20MiB (28526336 bytes) from device: RESOURCE_EXHAUSTED: : CUDA_ERROR_OUT_OF_MEMORY: out of memory
I0000 00:00:1771086795.720009   54572 cuda_executor.cc:508] failed to allocate 24.48MiB (25673728 bytes) from device: RESOURCE_EXHAUSTED: : CUDA_ERROR_OUT_OF_MEMORY: out of

In [ ]:
df.shape